# Pizza Hut Reviews
---

## Step-1 Gather Data

<ul>
    <li>Source: <a href="https://www.consumeraffairs.com/food/pizza-hut.html">Pizza Hut Reviews</a> </li>
    <li>Collect User Review, Ratings and Upvotes from all reviews </li>
</ul>

In [225]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

source_link = "https://www.consumeraffairs.com/food/pizza-hut.html?page="

In [226]:
initial_page = source_link+"1"
html_doc = requests.get(source_link).text
soup = BeautifulSoup(html_doc, 'html.parser')

In [227]:
# Find pagination first and last index and reviews container
pagination_start = 1
pagination_end = int(soup.find('nav',class_="pgn").find('a',class_="pgn__btn--last")["href"].split("#")[0].split("=")[1])


In [279]:
# Go through reviews and parse required data 

def parse_reviews(reviews):
    parsed_reviews = []
    for review in reviews:
        try:
            review_obj = {}
            review_obj['review_rating'] = review.select_one('.rvw__hdr').select_one('.rvw__hdr-stat').find_all('meta')[1]['content'];
            review_obj['review_body'] = review.select_one('.rvw-bd').find('p').text
            upvote_elem = review.select_one('.rvw-foot').find('span').select_one('strong')
            review_obj['review_upvotes'] = int(upvote_elem.text.split(" ")[0]) if upvote_elem != None else 0
            review_obj['review_created_at']= review.select_one('.rvw-bd').find('span').text.split(": ")[1]
            parsed_reviews.append(review_obj)
        except Exception as e:
            pass
    return parsed_reviews


In [280]:
# Visit all the pages except page 1 as we already have it parsed and store all the reviews from each page in a list
reviews = []

for i in range(pagination_start, pagination_end + 1):
    if i == 1:
        reviews_container = soup.find_all('div', class_="rvw")
        reviews.extend(parse_reviews(reviews_container))        
    else:
        page_link = initial_page = source_link+str(i)
        print("Scraping {}".format(page_link))
        page_html = requests.get(page_link).text
        page_parsed = BeautifulSoup(page_html, 'html.parser')
        page_reviews_container = page_parsed.find_all('div', class_="rvw")
        page_reviews_parsed = parse_reviews(page_reviews_container)
        reviews.extend(page_reviews_parsed)
        
        

print("Total Reviews {}".format(str(len(reviews))))

Total Reivews  10
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=2
Total Reivews  30
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=3
Total Reivews  30
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=4
Total Reivews  30
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=5
Total Reivews  30
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=6
Total Reivews  30
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=7
Total Reivews  30
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=8
Total Reivews  30
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=9
Total Reivews  30
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=10
Total Reivews  30
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=11
Total Reivews  30
Scraping https://www.consumeraffairs.com/food/pizza-hut.html?page=12
Total Reivews  30
Scraping https://www.consumeraffa

In [296]:
print("Total Reviews:",len(reviews))


Total Reviews: 605


## Step-2 Persist Data
---
<li>Store Data in a <b>SQLite</b> file</li>

In [293]:
import sqlite3

In [292]:
def save_sqlite(reviews):
    conn = sqlite3.connect("pizza_hut_reviews.db")
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS Tbl_Reviews (
        id integer primary key autoincrement, 
        review text, 
        created_at text, 
        ratings number,
        upvotes number
    )''')
    
    review_sql_payload_tuples = []
    for review in reviews:
        review_sql_payload_tuples.append(
            (review['review_body'],review['review_created_at'],review['review_rating'],review['review_upvotes'])
        )
    
    query = cur.executemany('INSERT INTO Tbl_Reviews (review, created_at, ratings, upvotes) VALUES (?,?,?,?)',review_sql_payload_tuples)

    conn.commit()
    conn.close()
    
    
save_sqlite(reviews)